In [1]:
import os
import math
import json
from openai import OpenAI

def generate_html_with_token_probs(response, output_filename="token_probabilities.html"):
    """
    Generate an interactive HTML file showing tokens with a side panel for probabilities.
    Uses actual token from logprobs data.
    
    Args:
        response: OpenAI API response with logprobs
        output_filename: Name of the HTML file to generate
    """
    
    choice = response.choices[0]
    logprobs_data = choice.logprobs.content
    
    # Build tokens data structure using actual tokens from logprobs
    tokens_data = []
    for i, logprobs in enumerate(logprobs_data):
        alternatives = []
        for logprob in logprobs.top_logprobs:
            token = logprob.token
            token_logprob = logprob.logprob
            prob = math.exp(token_logprob)
            alternatives.append({
                "token": token,
                "logprob": token_logprob,
                "probability": prob
            })
        
        # Get the actual token that was chosen by the LLM
        actual_token = logprobs.token
        
        tokens_data.append({
            "index": i,
            "token": actual_token,
            "alternatives": alternatives
        })
    
    # Generate HTML
    html_content = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Token Probability Visualization</title>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        body {{
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, sans-serif;
            background: #f5f7fa;
            height: 100vh;
            display: flex;
            overflow: hidden;
        }}
        
        .left-panel {{
            flex: 0 0 60%;
            overflow-y: auto;
            padding: 40px;
            background: white;
            border-right: 1px solid #e0e0e0;
        }}
        
        .right-panel {{
            flex: 0 0 40%;
            background: #f5f7fa;
            padding: 40px;
            overflow-y: auto;
            border-left: 1px solid #e0e0e0;
        }}
        
        h1 {{
            color: #2c3e50;
            margin-bottom: 30px;
            font-size: 28px;
        }}
        
        .text-content {{
            font-size: 18px;
            line-height: 1.8;
            color: #333;
            word-wrap: break-word;
            overflow-wrap: break-word;
            white-space: pre-wrap;
        }}
        
        .token {{
            position: relative;
            cursor: pointer;
            padding: 0;
            margin: 0;
            border-radius: 3px;
            transition: all 0.15s ease;
            background-color: transparent;
            border-bottom: 2px solid transparent;
            display: inline;
        }}
        
        .token:hover {{
            background-color: #fffacd;
            border-bottom-color: #ffd700;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }}
        
        .panel-title {{
            color: #2c3e50;
            font-size: 18px;
            font-weight: 600;
            margin-bottom: 20px;
            border-bottom: 2px solid #3498db;
            padding-bottom: 10px;
        }}
        
        .current-token {{
            background: #3498db;
            color: white;
            padding: 12px 16px;
            border-radius: 6px;
            font-size: 16px;
            font-weight: 600;
            margin-bottom: 20px;
            text-align: center;
            word-break: break-all;
        }}
        
        .placeholder {{
            color: #999;
            font-style: italic;
            text-align: center;
            padding: 40px 20px;
        }}
        
        .token-option {{
            margin-bottom: 16px;
            padding: 12px;
            background: white;
            border-radius: 6px;
            border-left: 4px solid #3498db;
            transition: all 0.2s ease;
        }}
        
        .token-option.selected {{
            border-left-color: #e74c3c;
            background: #fff5f5;
        }}
        
        .rank {{
            display: inline-block;
            background: #3498db;
            color: white;
            width: 28px;
            height: 28px;
            border-radius: 50%;
            text-align: center;
            line-height: 28px;
            font-size: 12px;
            font-weight: 600;
            margin-right: 8px;
        }}
        
        .token-option.selected .rank {{
            background: #e74c3c;
        }}
        
        .token-name {{
            font-weight: 600;
            color: #2c3e50;
            margin-bottom: 6px;
            display: flex;
            align-items: center;
            font-size: 14px;
        }}
        
        .probability-bar {{
            background: #ecf0f1;
            height: 8px;
            border-radius: 4px;
            overflow: hidden;
            margin: 6px 0;
        }}
        
        .probability-fill {{
            background: linear-gradient(90deg, #2ecc71, #27ae60);
            height: 100%;
            border-radius: 4px;
            transition: width 0.3s ease;
        }}
        
        .token-option.selected .probability-fill {{
            background: linear-gradient(90deg, #e74c3c, #c0392b);
        }}
        
        .probability-text {{
            font-size: 12px;
            color: #666;
            margin-top: 4px;
            display: flex;
            justify-content: space-between;
        }}
        
        @media (max-width: 1024px) {{
            .left-panel {{
                flex: 0 0 50%;
            }}
            .right-panel {{
                flex: 0 0 50%;
            }}
        }}
        
        @media (max-width: 768px) {{
            body {{
                flex-direction: column;
            }}
            .left-panel {{
                flex: 0 0 50%;
                border-right: none;
                border-bottom: 1px solid #e0e0e0;
            }}
            .right-panel {{
                flex: 0 0 50%;
                border-left: none;
            }}
        }}
    </style>
</head>
<body>
    <div class="left-panel">
        <h1>Token Probabilities</h1>
        <div class="text-content" id="textContent"></div>
    </div>
    
    <div class="right-panel">
        <div class="panel-title">Alternatives</div>
        <div id="probabilitiesPanel" class="placeholder">
            Hover over a token to see alternatives
        </div>
    </div>
    
    <script>
        const tokensData = {json.dumps(tokens_data)};
        
        function createTokens() {{
            const textContainer = document.getElementById('textContent');
            
            // Create token spans based on actual token boundaries
            tokensData.forEach((tokenData, index) => {{
                const tokenSpan = document.createElement('span');
                tokenSpan.className = 'token';
                tokenSpan.textContent = tokenData.token;
                
                // Add hover handler
                tokenSpan.addEventListener('mouseenter', () => {{
                    updateProbabilitiesPanel(index);
                }});
                
                textContainer.appendChild(tokenSpan);
            }});
        }}
        
        function updateProbabilitiesPanel(tokenIndex) {{
            const panel = document.getElementById('probabilitiesPanel');
            const alternatives = tokensData[tokenIndex].alternatives;
            const currentToken = tokensData[tokenIndex].token;
            
            let html = '<div class="current-token">Selected: ' + escapeHtml(currentToken) + '</div>';
            
            alternatives.forEach((alt, idx) => {{
                const percentage = (alt.probability * 100).toFixed(2);
                const barWidth = (alt.probability * 100);
                const isSelected = alt.token === currentToken ? 'selected' : '';
                html += `
                    <div class="token-option ${{isSelected}}">
                        <div class="token-name">
                            <span class="rank">${{idx + 1}}</span>
                            <span>${{escapeHtml(alt.token)}}</span>
                        </div>
                        <div class="probability-bar">
                            <div class="probability-fill" style="width: ${{barWidth}}%;"></div>
                        </div>
                        <div class="probability-text">
                            <span>Log: ${{alt.logprob.toFixed(4)}}</span>
                            <span>${{percentage}}%</span>
                        </div>
                    </div>
                `;
            }});
            
            panel.innerHTML = html;
        }}
        
        function escapeHtml(text) {{
            const map = {{
                '&': '&amp;',
                '<': '&lt;',
                '>': '&gt;',
                '"': '&quot;',
                "'": '&#039;'
            }};
            return text.replace(/[&<>"']/g, m => map[m]);
        }}
        
        createTokens();
    </script>
</body>
</html>"""
    
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"HTML file generated: {output_filename}")


# Main usage
if __name__ == "__main__":
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "user", "content": "Tell me a story."}
        ],
        logprobs=True,
        top_logprobs=10,
    )

    # Generate the HTML file
    generate_html_with_token_probs(response, "token_probabilities.html")
    
    # Print to console as well
    choice = response.choices[0]
    print("Generated text:")
    print(choice.message.content)
    print("\nHTML file created successfully!")

HTML file generated: token_probabilities.html
Generated text:
Once upon a time, in a small village nestled between rolling green hills and a shimmering blue lake, lived a curious girl named Elara. She had an insatiable thirst for adventure and a heart full of dreams. Every day, she would explore the forests nearby, imagining that she was discovering lost kingdoms and magical creatures.

One morning, while wandering deeper into the woods than ever before, Elara stumbled upon an ancient, moss-covered door hidden among the roots of a great oak tree. Intrigued, she pushed it open and found a narrow staircase spiraling down into the earth. Gathering her courage, she descended the steps, which led to a vast underground cavern filled with glowing crystals and sparkling waterfalls.

In the center of the cavern stood a majestic, silver stag with eyes that seemed to hold the wisdom of centuries. The stag spoke softly, telling Elara that she had been chosen to protect the secret of the enchanted 

In [2]:
choice

Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Once', bytes=[79, 110, 99, 101], logprob=-0.413409560918808, top_logprobs=[TopLogprob(token='Once', bytes=[79, 110, 99, 101], logprob=-0.413409560918808), TopLogprob(token='Sure', bytes=[83, 117, 114, 101], logprob=-1.4134095907211304), TopLogprob(token='Certainly', bytes=[67, 101, 114, 116, 97, 105, 110, 108, 121], logprob=-2.413409471511841), TopLogprob(token='Of', bytes=[79, 102], logprob=-5.16340970993042), TopLogprob(token='Absolutely', bytes=[65, 98, 115, 111, 108, 117, 116, 101, 108, 121], logprob=-9.663409233093262), TopLogprob(token='In', bytes=[73, 110], logprob=-14.288409233093262), TopLogprob(token='There', bytes=[84, 104, 101, 114, 101], logprob=-16.038410186767578), TopLogprob(token=' once', bytes=[32, 111, 110, 99, 101], logprob=-16.663410186767578), TopLogprob(token='Here', bytes=[72, 101, 114, 101], logprob=-16.913410186767578), TopLogprob(token='Long', bytes=[76, 1